In [1]:
from azureml.core import Workspace, Datastore, Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

In [2]:
pip install azureml-pipeline

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Initialize workspace
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json")

# Define compute target
compute_name = "OnlineRetail"
compute_target = ws.compute_targets[compute_name]

In [4]:
# Define environment
env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)


{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
       

In [27]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - joblib
  - scikit-learn



In [28]:
# Define run configuration
aml_config = RunConfiguration()
aml_config.target = compute_target
aml_config.environment = env

In [43]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

source_directory = ""

#Script initialization
py_script_run_read = PythonScriptStep(
                name='Data Wrangling Step',
                script_name = read_data,
                compute_target = compute_target,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

In [44]:
# Experiment

# pipeline_run = Experiment(ws, "First_run_7").submit(pipeline_1)
# pipeline_run.wait_for_completion(show_output=True)

experiment_name = "Run3"
experiment = Experiment(ws, experiment_name)
pipeline_run = experiment.submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

SnapshotException: SnapshotException:
	Message: ====================================================================

While attempting to take snapshot of /mnt/batch/tasks/shared/LS_root/mounts/clusters/onlineretail/code/Users/mypersonall3099/Online_retail
Your total snapshot size exceeds the limit of 300.0 MB.
Please see http://aka.ms/aml-largefiles on how to work with large files.

====================================================================


	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "====================================================================\n\nWhile attempting to take snapshot of /mnt/batch/tasks/shared/LS_root/mounts/clusters/onlineretail/code/Users/mypersonall3099/Online_retail\nYour total snapshot size exceeds the limit of 300.0 MB.\nPlease see http://aka.ms/aml-largefiles on how to work with large files.\n\n====================================================================\n\n"
    }
}

In [32]:
# Experiment 2

pipeline_run = Experiment(ws, "Second_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun 42601c31-040e-4aac-91b3-8ed00e0e80fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42601c31-040e-4aac-91b3-8ed00e0e80fa?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 42601c31-040e-4aac-91b3-8ed00e0e80fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42601c31-040e-4aac-91b3-8ed00e0e80fa?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 0592e3a7-ddda-4f2d-8279-8df845a155d9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0592e3a7-ddda-4f2d-8279-8df845a155d9?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
StepRun( Data Wrangling 

'Finished'

In [33]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
Second_run,42601c31-040e-4aac-91b3-8ed00e0e80fa,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [39]:
from azureml.core import Workspace, Datastore

# Get the workspace
ws = Workspace.from_config()

# Access the datastore
datastore = Datastore.get(ws, 'workspaceblobstore')

# Define the target path and prefix
target_path = "azureml"  # Local path to save the file
prefix = "azureml/ExperimentRun/dcid.b4f0444e-02f9-4cad-84c9-80ad6ac4e399/outputs/best_autoarima_model.pkl"

# Download the model file from the datastore
datastore.download(target_path=target_path, prefix=prefix, overwrite=False, show_progress=True)


0